<a href="https://colab.research.google.com/github/ryan-saloma/bike-market-summit/blob/main/Bike_Market_Summit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U googlemaps

import googlemaps as gm
import requests
import pandas as pd
import plotly
import plotly.express as px

gmaps = gm.Client(key='XXXXXXXXXX')

# First get location of Summit, NJ
def get_coordinates_from_address(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': address,
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get('results')
        if results:
            location = results[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("No results found for the given city name.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

summit_trek_coords = \
get_coordinates_from_address('348 Springfield Ave, Summit, NJ 07901', gmaps.key)

# How far would someone travel for a bike?
# According to a bike forum, some people are willing to drive 2+ hours
# source: https://www.bikeforums.net/road-cycling/947577-how-far-you-willing-drive-buy-new-bike.html
#
# According to one study, people typically don't travel more than 20 min
# for routine purchases. This suggests the market for bikes and the market for
# bike accessories are different, as people will travel further for expensive
# purchases and unique experiences (classes, etc.)
#
# source: https://blog.accessdevelopment.com/research-how-far-will-consumers-travel-to-make-routine-purchases
#
# TODO: work the other direction and figure out how large an area you'd need to
# reach to meet sales goals

bike_stores = gmaps.places(query='bike store', \
                                 location=summit_trek_coords,radius=10000, \
                                 type='bicycle_store')
# Get results as a list of dicts
results = bike_stores['results']

# Perform list comprehension to extract keys of interest
keys_of_interest = ['name', 'rating', 'user_ratings_total', 'geometry']
stores_df = pd.DataFrame(results)
stores_df

# Get coords of competing stores
coords_competition = pd.DataFrame()
coords_competition['Name'] = stores_df['name']
coords_competition['Latitude'] = stores_df['geometry'].apply(lambda x: x['location']['lat'])
coords_competition['Longitude'] = stores_df['geometry'].apply(lambda x: x['location']['lng'])
coords_competition['Rating'] = stores_df['rating']
coords_competition['Number of Ratings'] = stores_df['user_ratings_total']
coords_competition

coords_competition.dropna(
    axis=0,
    how='any',
    subset=None,
    inplace=True
)

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(coords_competition,
                        lat="Latitude",
                        lon="Longitude",
                        hover_name="Name",
                        hover_data=["Name", "Rating"],
                        color="Rating",
                        color_continuous_scale=color_scale,
                        size="Number of Ratings",
                        zoom=8,
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
plotly.io.write_html(fig, "competition_map.html")


## Market Analysis

Sources:
- https://www.businessnewsdaily.com/15751-conduct-market-analysis.html
- https://www.sba.gov/business-guide/plan-your-business/market-research-competitive-analysis

Who are my potential customers?
What are my customers' buying habits?
How large is my target market?
How much are customers willing to pay for my product?
Who are my main competitors?
What are my competitors' strengths and weaknesses?

To-Do List
- List products sold at Trek Store, list products made by Trek, disjoint set
- Get reviews from competitors, create word cloud

In [ ]:
# Product Information

%mkdir scraper
%pip install bs4

from bs4 import BeautifulSoup

# Set base URL
base_url = "https://www.trekbikes.com/us/en_US/retail/summit/"
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0',
})
proxies = {
    'http':'XXXXXXXXXXXXXXX'
}

#
k = requests.get(base_url, headers=headers, proxies=proxies).text
soup = BeautifulSoup(k,'html.parser')
product_list = soup.find_all("div", {"class": "cat-product-thumb-container"})
product_list

# It looks like the content might be dynamically loaded, given the results
# We need to use Selenium instead

In [56]:
%pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(chrome_options)
driver.get('https://www.trekbikes.com/us/en_US/bikes/c/B100/')
all_links = []

def extract_links_from_page(driver):
    # Extract all the links on the current page
    links = driver.find_elements(By.TAG_NAME, 'a')
    return [link.get_attribute('href') for link in links if link.get_attribute('href')]

while True:
    # Extract links from the current page
    all_links.extend(extract_links_from_page(driver))

    # Check if there's a "Next" button to go to the next page
    try:
        next_button = driver.find_element(By.ID, 'search-page-next')  # Adjust this locator as needed
        if next_button.get_attribute('class') != 'pagination__button arrow-right-icon--weak disabled':
            next_button.click()
        else:
            break
    except:
        break

unique_links = set(all_links)
links_series = pd.Series(list(unique_links))
links_series.to_csv('links_to_bikes.csv')